**Import Libraries**

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pydot
import os
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import sklearn
from sklearn.metrics import confusion_matrix

**CSV Reading**

In [ ]:
train_dir = '../input/tabular-playground-series-may-2021/train.csv'
test_dir = '../input/tabular-playground-series-may-2021/test.csv'

In [ ]:
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
test_id = pd.read_csv(test_dir)

In [ ]:
train_df.head()
test_df.head()

**Data Preparation**

In [ ]:
train_df = train_df.drop(['id'], axis=1)

In [ ]:
test_df = test_df.drop(['id'], axis=1)

In [ ]:
train_df_features = train_df.copy()
train_df_labels = train_df_features.pop('target')

In [ ]:
label_dict = {'Class_1': 0,'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
train_df_labels = [label_dict[item] for item in train_df_labels]

In [ ]:
train_df_labels

In [ ]:
train_df_features = np.array(train_df_features)
train_df_labels = np.array(train_df_labels)

In [ ]:
train_df_labels = to_categorical(train_df_labels)

In [ ]:
num_classes = 4
num_features = 50

**Model**

In [ ]:
def dnn_model():
    input_layer = Input(shape=(num_features,))
    normalize_layer = tf.keras.layers.experimental.preprocessing.Normalization()(input_layer)

    #Layer 1
    dense_1 = Dense(1024, activation='relu')(normalize_layer)
    batch_norm_1 = BatchNormalization()(dense_1)

    dense_2 = Dense(512, activation='relu')(batch_norm_1)
    batch_norm_2 = BatchNormalization()(dense_2)
    drop_1 = Dropout(0.1)(batch_norm_2)

    dense_3 = Dense(256, activation='relu')(drop_1)
    batch_norm_3 = BatchNormalization()(dense_3)

    dense_4 = Dense(128, activation='relu')(batch_norm_3)
    batch_norm_4 = BatchNormalization()(dense_4)
    drop_2 = Dropout(0.1)(batch_norm_4)

    #Layer 2
    dense_1_2 = Dense(1024, activation='relu')(normalize_layer)
    batch_norm_1_2 = BatchNormalization()(dense_1_2)

    dense_2_2 = Dense(512, activation='relu')(batch_norm_1_2)
    batch_norm_2_2 = BatchNormalization()(dense_2_2)
    drop_1_2 = Dropout(0.2)(batch_norm_2_2)

    dense_3_2 = Dense(512, activation='relu')(drop_1_2)
    batch_norm_3_2 = BatchNormalization()(dense_3_2)

    dense_4_2 = Dense(256, activation='relu')(batch_norm_3_2)
    batch_norm_4_2 = BatchNormalization()(dense_4_2)

    dense_5_2 = Dense(128, activation='relu')(batch_norm_4_2)
    batch_norm_5_2 = BatchNormalization()(dense_5_2)
    drop_2_2 = Dropout(0.2)(batch_norm_5_2)

    #Layer 3
    dense_1_3 = Dense(1024, activation='relu')(normalize_layer)
    batch_norm_1_3 = BatchNormalization()(dense_1_3)
    drop_1_3 = Dropout(0.1)(batch_norm_1_3)

    dense_2_3 = Dense(512, activation='relu')(batch_norm_1_3)
    batch_norm_2_3 = BatchNormalization()(dense_2_3)
    drop_2_3 = Dropout(0.1)(batch_norm_2_3)

    dense_3_3 = Dense(256, activation='relu')(drop_1_3)
    batch_norm_3_3 = BatchNormalization()(dense_3_3)
    drop_3_3 = Dropout(0.1)(batch_norm_3_3)

    dense_4_3 = Dense(256, activation='relu')(batch_norm_3_3)
    batch_norm_4_3 = BatchNormalization()(dense_4_3)
    drop_4_3 = Dropout(0.1)(batch_norm_4_3)

    dense_5_3 = Dense(128, activation='relu')(batch_norm_4_3)
    batch_norm_5_3 = BatchNormalization()(dense_5_3)
    drop_5_3 = Dropout(0.1)(batch_norm_5_3)

    #Concat
    concat = layers.concatenate([drop_2, drop_2_2, drop_5_3, normalize_layer])

    #Final Layer
    dense_f1 = Dense(512, activation='relu')(concat)
    batch_norm_f1 = BatchNormalization()(dense_f1)
    drop_f1 = Dropout(0.1)(batch_norm_f1)
    
    dense_f2 = Dense(256, activation='relu')(drop_f1)
    batch_norm_f2 = BatchNormalization()(dense_f2)
    
    dense_f3 = Dense(128, activation='relu')(batch_norm_f2)
    batch_norm_f3 = BatchNormalization()(dense_f3)
    drop_f2 = Dropout(0.1)(batch_norm_f3)

    dense_f4 = Dense(64, activation='relu')(drop_f2)
    batch_norm_f4 = BatchNormalization()(dense_f4)

    final = Dense(num_classes, activation='relu')(batch_norm_f4)

    model = Model(inputs = input_layer, outputs = final)
    return model
    

In [ ]:
model = dnn_model()
model.summary()

In [ ]:
model.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.001), metrics=['accuracy'])

**Training**

In [ ]:
training = model.fit(train_df_features, train_df_labels, batch_size=200, validation_split=0.2, epochs=10)

**Predictions**

In [ ]:
preds = pd.DataFrame(model.predict(test_df))
preds.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4']
preds['id'] = test_id['id']
preds = preds[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]

preds.to_csv('tps_may_submission', index=False)
preds

Go check out https://www.kaggle.com/soyabulislam/tps-may-tf-functional-api-approach as well if you want as this notebook also uses the tensorflow functional approach to solve this problem.